# Confluence Connector Setup for Amazon Q Business using OAuth Permissions
## Simple OAuth token generation and Q Business integration

This notebook provides a streamlined approach to setting up Confluence Cloud integration with Amazon Q Business.

What you'll accomplish:
- Generate Confluence OAuth tokens with minimal setup
- Store tokens securely in AWS Secrets Manager
- Create Q Business application and Confluence data source
- Test the integration end-to-end

## ⚠️ IMPORTANT: Atlassian OAuth App Setup Required

**Before running the cells below, you must create an OAuth app in Atlassian Developer Console:**

1. 🔗 **Go to**: [Atlassian Developer Console](https://developer.atlassian.com/console/myapps/)
2. 📱 **Create**: New OAuth 2.0 (3LO) app
3. ⚙️ **Configure**: 
   - **App name**: Choose any name (e.g., "Q Business Confluence Integration")
   - **Callback URL**: Use the value from your `.env` file (`CONFLUENCE_CALLBACK_URL`)
   - **Permissions**: Will be handled automatically by this notebook
4. 📋 **Copy**: Client ID and Client Secret to your `.env` file

📖 **Detailed setup guide**: [Q Business User Guide - Confluence OAuth Setup](https://docs.aws.amazon.com/amazonq/latest/qbusiness-ug/confluence-cloud-credentials-oauth.html)

## Step 1: Install Required Packages

In [ ]:
# Install required packages
!pip install boto3 requests python-dotenv

import boto3
import json
import requests
import secrets
import string
import time
import urllib.parse

## Step 2: Configuration and Environment Variables

This step loads configuration from environment variables and generates unique resource names:

In [ ]:
import os
from pathlib import Path


# Generate a random suffix for unique naming
def generate_suffix(length=5):
    return "".join(
        secrets.choice(string.ascii_lowercase + string.digits) for _ in range(length)
    )


# Try to load python-dotenv if available
try:
    from dotenv import load_dotenv

    load_dotenv()  # Load .env file from current directory
    print("✅ Loaded environment variables from .env file")
except ImportError:
    print("💡 python-dotenv not installed. Install with: pip install python-dotenv")
    print("   Or set environment variables manually in your shell")
    print("   Continuing with system environment variables...")

# Load configuration from environment variables
confluence_domain = os.getenv("CONFLUENCE_DOMAIN")
client_id = os.getenv("CONFLUENCE_CLIENT_ID")
client_secret = os.getenv("CONFLUENCE_CLIENT_SECRET")
region = os.getenv("AWS_REGION", "us-west-2")  # Default to us-west-2
callback_url = os.getenv(
    "CONFLUENCE_CALLBACK_URL", "http://localhost:8080/callback"
)  # Default to localhost

# Validate required environment variables
required_vars = {
    "CONFLUENCE_DOMAIN": confluence_domain,
    "CONFLUENCE_CLIENT_ID": client_id,
    "CONFLUENCE_CLIENT_SECRET": client_secret,
}

missing_vars = [var for var, value in required_vars.items() if not value]
if missing_vars:
    print("❌ Missing required environment variables:")
    for var in missing_vars:
        print(f"   - {var}")
    print("\n💡 Please set these in your .env file or system environment")
    print("   Copy .env.example to .env and update with your values")
    raise ValueError(
        f"Missing required environment variables: {', '.join(missing_vars)}"
    )

# Generate unique resource names
suffix = generate_suffix()
app_name = f"confluence-qbusiness-{suffix}"
index_name = f"confluence-index-{suffix}"
datasource_name = f"confluence-datasource-{suffix}"
secret_name = f"confluence-oauth-tokens-{suffix}"

# AWS credentials are automatically picked up by boto3 from:
# AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY, AWS_SESSION_TOKEN (if using temporary credentials)
# Or from AWS CLI configuration, IAM roles, etc.

print(f"Using app name: {app_name}")
print(f"Using secret name: {secret_name}")
print(f"Using Confluence domain: {confluence_domain}")
print(f"Using AWS region: {region}")

## Step 3: Validate AWS Credentials

In [ ]:
# Validate AWS credentials
try:
    sts = boto3.client("sts", region_name=region)
    identity = sts.get_caller_identity()

    print("✅ AWS credentials validated!")
    print(f"Account ID: {identity['Account']}")
    print(f"User ARN: {identity['Arn']}")

    account_id = identity["Account"]

except Exception as e:
    print(f"❌ AWS credentials error: {str(e)}")
    print("Please run 'aws configure' to set up your credentials")

## Step 4: Generate OAuth Authorization URL

This step creates the authorization URL that you'll visit in your browser to grant permissions to your OAuth app. The URL includes all the necessary Confluence scopes optimized for Q Business integration, ensuring comprehensive access to pages, spaces, users, and metadata.

In [ ]:
# OAuth Scopes - Comprehensive Confluence permissions for Q Business
oauth_scopes = [
    "read:me",
    "read:account",
    "read:content:confluence",
    "read:content-details:confluence",
    "read:space-details:confluence",
    "read:audit-log:confluence",
    "read:page:confluence",
    "read:attachment:confluence",
    "read:blogpost:confluence",
    "read:custom-content:confluence",
    "read:comment:confluence",
    "read:template:confluence",
    "read:label:confluence",
    "read:watcher:confluence",
    "read:group:confluence",
    "read:relation:confluence",
    "read:user:confluence",
    "read:configuration:confluence",
    "read:space:confluence",
    "read:space.permission:confluence",
    "read:space.property:confluence",
    "read:user.property:confluence",
    "read:space.setting:confluence",
    "read:analytics.content:confluence",
    "read:content.permission:confluence",
    "read:content.property:confluence",
    "read:content.restriction:confluence",
    "read:content.metadata:confluence",
    "read:inlinetask:confluence",
    "read:task:confluence",
    "read:permission:confluence",
    "read:whiteboard:confluence",
    "read:database:confluence",
    "read:embed:confluence",
    "read:folder:confluence",
    "read:app-data:confluence",
    "read:email-address.summary",
    "offline_access",
]


# Generate OAuth authorization URL
def generate_auth_url(client_id, callback_url, scopes):
    base_url = "https://auth.atlassian.com/authorize"

    params = {
        "audience": "api.atlassian.com",
        "client_id": client_id,
        "scope": " ".join(scopes),
        "redirect_uri": callback_url,
        "state": "from_notebook",
        "response_type": "code",
        "prompt": "consent",
    }

    return f"{base_url}?{urllib.parse.urlencode(params)}"


# Generate the authorization URL
auth_url = generate_auth_url(client_id, callback_url, oauth_scopes)

print("🔗 Authorization URL:")
print("=" * 80)
print(auth_url)
print("=" * 80)
print("📋 Instructions:")
print("1. Click the URL above (or copy/paste into browser)")
print("2. Log in to Atlassian and authorize the app")
print("3. Copy the 'code' parameter from the redirect URL")
print("4. Paste it in the next cell")

## Step 5: Exchange Authorization Code for Tokens

After you authorize the app in your browser, you'll receive an authorization code. This step exchanges that temporary code for long-lived access tokens that Q Business will use to authenticate with Confluence. The process also generates a refresh token for automatic token renewal.

In [ ]:
# Paste your authorization code here
authorization_code = "YOUR_CODE_HERE"


# Exchange code for tokens
def exchange_code_for_tokens(client_id, client_secret, code, callback_url):
    token_url = "https://auth.atlassian.com/oauth/token"

    data = {
        "grant_type": "authorization_code",
        "client_id": client_id,
        "client_secret": client_secret,
        "code": code,
        "redirect_uri": callback_url,
    }

    response = requests.post(token_url, data=data)

    if response.status_code == 200:
        return response.json()
    else:
        print(f"❌ Token exchange failed: {response.status_code}")
        print(f"Response: {response.text}")
        return None


# Exchange the code for tokens
if authorization_code != "PASTE_YOUR_CODE_HERE":
    tokens = exchange_code_for_tokens(
        client_id, client_secret, authorization_code, callback_url
    )

    if tokens:
        print("✅ Tokens obtained successfully!")
        print(f"Access token: {tokens['access_token'][:20]}...")
        print(f"Refresh token: {tokens['refresh_token'][:20]}...")

        access_token = tokens["access_token"]
        refresh_token = tokens["refresh_token"]
    else:
        print("❌ Failed to obtain tokens")
else:
    print("⚠️ Please paste your authorization code above")

## Step 6: Test Confluence API Access

This step validates that your OAuth tokens work correctly by making a test API call to Confluence. It retrieves your accessible Confluence sites and confirms that the tokens have the proper permissions. This ensures the data source will be able to connect successfully.

In [ ]:
# Test Confluence API access
def test_confluence_access(domain, access_token):
    # Get accessible resources (sites)
    resources_url = "https://api.atlassian.com/oauth/token/accessible-resources"
    headers = {"Authorization": f"Bearer {access_token}", "Accept": "application/json"}

    response = requests.get(resources_url, headers=headers)

    if response.status_code == 200:
        resources = response.json()
        print(f"✅ Found {len(resources)} accessible resources")

        # Find the matching site
        site_id = None
        for resource in resources:
            if domain in resource.get("url", ""):
                site_id = resource["id"]
                print(f"✅ Found matching site: {resource['name']} ({resource['url']})")
                break

        if site_id:
            print(f"✅ Confluence API test successful!")
            return site_id
        else:
            print(f"❌ Could not find site matching domain: {domain}")
            return None
    else:
        print(f"❌ Failed to get accessible resources: {response.status_code}")
        return None


# Test the API access
if "access_token" in locals():
    site_id = test_confluence_access(confluence_domain, access_token)
else:
    print("⚠️ No access token available. Complete the previous steps first.")

## Step 7: Store Tokens in AWS Secrets Manager

AWS Secrets Manager provides secure, encrypted storage for your OAuth tokens. This step stores both the access token and refresh token along with your OAuth app credentials. Q Business will reference this secret to authenticate with Confluence during data source synchronization.

In [ ]:
# Store tokens in Secrets Manager
def store_tokens_in_secrets_manager(secret_name, tokens, confluence_config):
    secrets_client = boto3.client("secretsmanager", region_name=region)

    secret_value = {
        "confluenceAccessToken": tokens["access_token"],
        "confluenceRefreshToken": tokens["refresh_token"],
        "confluenceAppKey": confluence_config["client_id"],
        "confluenceAppSecret": confluence_config["client_secret"],
    }

    try:
        # Try to create the secret
        response = secrets_client.create_secret(
            Name=secret_name,
            Description="Confluence OAuth tokens for Q Business integration",
            SecretString=json.dumps(secret_value),
        )
        print(f"✅ Secret created: {response['ARN']}")
        return response["ARN"]

    except secrets_client.exceptions.ResourceExistsException:
        # Secret already exists, update it
        response = secrets_client.update_secret(
            SecretId=secret_name, SecretString=json.dumps(secret_value)
        )
        print(f"✅ Secret updated: {response['ARN']}")
        return response["ARN"]

    except Exception as e:
        print(f"❌ Failed to store secret: {str(e)}")
        return None


# Store the tokens
if "tokens" in locals() and "site_id" in locals():
    confluence_config = {
        "client_id": client_id,
        "client_secret": client_secret,
        "site_id": site_id,
        "domain": confluence_domain,
    }

    secret_arn = store_tokens_in_secrets_manager(secret_name, tokens, confluence_config)
else:
    print("⚠️ Missing tokens or site_id. Complete previous steps first.")

## Step 8: Create IAM Role for Q Business

Q Business requires an IAM service role to access AWS resources on your behalf. This role allows Q Business to read OAuth tokens from Secrets Manager, write indexed content to your index, and manage the data source synchronization process securely.

In [ ]:
# Create IAM role for Q Business
def create_qbusiness_role(role_name, account_id, region, secret_arn):
    iam = boto3.client("iam", region_name=region)

    # Trust policy for Q Business
    trust_policy = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Principal": {"Service": "qbusiness.amazonaws.com"},
                "Action": "sts:AssumeRole",
            }
        ],
    }

    # Permissions policy
    permissions_policy = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Action": ["secretsmanager:GetSecretValue"],
                "Resource": secret_arn,
            },
            {
                "Effect": "Allow",
                "Action": [
                    "qbusiness:BatchPutDocument",
                    "qbusiness:BatchDeleteDocument",
                ],
                "Resource": "*",
            },
        ],
    }

    try:
        # Create the role
        role_response = iam.create_role(
            RoleName=role_name,
            AssumeRolePolicyDocument=json.dumps(trust_policy),
            Description="Role for Q Business Confluence data source",
        )

        # Attach the permissions policy
        iam.put_role_policy(
            RoleName=role_name,
            PolicyName="ConfluenceDataSourcePermissions",
            PolicyDocument=json.dumps(permissions_policy),
        )

        role_arn = role_response["Role"]["Arn"]
        print(f"✅ IAM role created: {role_arn}")
        return role_arn

    except iam.exceptions.EntityAlreadyExistsException:
        # Role already exists
        role_response = iam.get_role(RoleName=role_name)
        role_arn = role_response["Role"]["Arn"]
        print(f"✅ Using existing IAM role: {role_arn}")
        return role_arn

    except Exception as e:
        print(f"❌ Failed to create IAM role: {str(e)}")
        return None


# Create the role
if "secret_arn" in locals():
    role_name = f"QBusiness-Confluence-Role-{suffix}"
    datasource_role_arn = create_qbusiness_role(
        role_name, account_id, region, secret_arn
    )
else:
    print("⚠️ No secret ARN available. Complete previous steps first.")

## Step 9: Create Q Business Application

The Q Business application is the main container for your AI-powered search and chat experience. It manages user access, integrates with your identity provider (AWS IAM or Identity Center), and coordinates all the components like indexes, data sources, and web experiences.

In [ ]:
# Find or create IAM Identity Center instance
def find_idc_instance():
    """Find existing IAM Identity Center instance or provide fallback options"""

    # List of regions to check for existing IDC instances
    regions_to_check = ["us-east-1", "us-west-2", "eu-west-1", "ap-southeast-2"]

    print("🔍 Looking for existing IAM Identity Center instances...")

    for check_region in regions_to_check:
        try:
            print(f"   Checking region: {check_region}")
            sso_admin = boto3.client("sso-admin", region_name=check_region)

            # List existing Identity Center instances
            response = sso_admin.list_instances()

            # Check if any instances exist in this region
            if response["Instances"]:
                instance_arn = response["Instances"][0]["InstanceArn"]
                instance_region = check_region
                print(
                    f"✅ Found IAM Identity Center instance in {check_region}: {instance_arn}"
                )
                return {
                    "identityType": "AWS_IAM_IDC",
                    "identityCenterInstanceArn": instance_arn,
                    "region": instance_region,
                }
        except Exception as e:
            # Continue to next region if there's an error
            print(f"   No access or instances in {check_region}")
            continue

    print("⚠️ No existing IAM Identity Center instances found.")
    print("💡 For production use, you should set up IAM Identity Center.")
    print("   For now, we'll use a simpler identity option.")

    # Return None to indicate we should use a simpler identity type
    return None


# Get IAM Identity Center configuration
idc_config = find_idc_instance()


# Create Q Business application
def create_qbusiness_application(app_name, idc_config=None):
    qbusiness = boto3.client("qbusiness", region_name=region)

    try:
        print(f"🚀 Creating Q Business application: {app_name}")

        # Prepare application parameters
        app_params = {
            "displayName": app_name,
            "description": "Confluence integration with Amazon Q Business",
        }

        # Add identity configuration if available
        if idc_config:
            app_params["identityType"] = idc_config["identityType"]
            if "identityCenterInstanceArn" in idc_config:
                app_params["identityCenterInstanceArn"] = idc_config[
                    "identityCenterInstanceArn"
                ]
            print(f"✅ Using Identity Center: {idc_config['identityType']}")
        else:
            # Use AWS_IAM as fallback
            app_params["identityType"] = "AWS_IAM"
            print("✅ Using AWS IAM identity type (fallback)")

        response = qbusiness.create_application(**app_params)

        application_id = response["applicationId"]
        print(f"✅ Application created: {application_id}")
        return application_id

    except Exception as e:
        print(f"❌ Failed to create application: {str(e)}")
        return None


# Create the application
application_id = create_qbusiness_application(app_name, idc_config)

if application_id:
    print(f"\n🎉 Q Business Application Created Successfully!")
    print(f"Application ID: {application_id}")
else:
    print("\n❌ Failed to create Q Business application")

## Step 10: Create Q Business Index

The index stores and organizes your content for fast, intelligent search. Q Business uses advanced AI to understand document relationships, extract key information, and enable semantic search capabilities. This is where your Confluence content will be processed and made searchable.

In [ ]:
# Create Q Business Index
def create_qbusiness_index(app_id, index_name):
    qbusiness = boto3.client("qbusiness", region_name=region)

    try:
        print(f"📚 Creating index: {index_name}")
        index_response = qbusiness.create_index(
            applicationId=app_id,
            displayName=index_name,
            description="Index for Confluence content",
        )

        index_id = index_response["indexId"]
        print(f"✅ Index created: {index_id}")

        # Wait for index to become active
        print("⏳ Waiting for index to become active...")
        print("   This usually takes 2-5 minutes")

        for attempt in range(20):  # Wait up to 10 minutes
            try:
                index_status = qbusiness.get_index(
                    applicationId=app_id, indexId=index_id
                )

                status = index_status.get("status", "UNKNOWN")
                print(f"   Index status: {status}")

                if status == "ACTIVE":
                    print(f"✅ Index is now active and ready!")
                    return index_id
                elif status in ["FAILED", "DELETING"]:
                    print(f"❌ Index creation failed with status: {status}")
                    return None
                else:
                    print(f"   Still creating... (attempt {attempt + 1}/20)")
                    time.sleep(30)  # Wait 30 seconds between checks

            except Exception as e:
                print(f"   Error checking status: {e}")
                time.sleep(30)

        # Final status check
        try:
            final_status = qbusiness.get_index(applicationId=app_id, indexId=index_id)
            final_state = final_status.get("status", "UNKNOWN")
            print(f"⚠️ Index creation timed out. Final status: {final_state}")
            print(f"   You can check status manually in AWS Console")
            print(f"   Index ID: {index_id}")
            return index_id  # Return ID even if not active yet
        except:
            print(f"⚠️ Could not get final status. Index ID: {index_id}")
            return index_id

    except Exception as e:
        print(f"❌ Failed to create index: {str(e)}")
        return None


# Create the index
if "application_id" in locals() and application_id:
    index_id = create_qbusiness_index(application_id, index_name)

    if index_id:
        print(f"\n🎉 Index Created Successfully!")
        print(f"Index ID: {index_id}")
        print("\n📋 Next Steps:")
        print("1. Proceed to create the Confluence data source")
        print("2. The index is now ready to receive content")
    else:
        print("\n❌ Failed to create index")
        print("Check the error messages above and retry if needed")
else:
    print("⚠️ No application_id available. Complete previous steps first.")

## Step 11: Create Confluence Data Source

The Confluence data source connector crawls your Confluence spaces and indexes the content for Q Business. This configuration includes OAuth credentials, crawl settings, and content filters. You can customize which spaces to include/exclude, content types to crawl, and sync schedules.

For advanced configuration options, see the [Amazon Q Business Confluence connector documentation](https://docs.aws.amazon.com/amazonq/latest/business-use-dg/confluence-connector.html).

In [ ]:
# Check if index is active before creating data source
if "index_id" in locals() and index_id:
    qbusiness = boto3.client("qbusiness", region_name=region)
    try:
        index_status = qbusiness.get_index(
            applicationId=application_id, indexId=index_id
        )
        status = index_status.get("status", "UNKNOWN")
        print(f"📋 Index status check: {status}")
        if status != "ACTIVE":
            print(f"⚠️ Index is not active yet (status: {status})")
            print(
                f"   Please wait for index to become ACTIVE before creating data source"
            )
            print(f"   You can re-run this cell once the index is ready")
        else:
            print(f"✅ Index is active and ready for data source creation")
    except Exception as e:
        print(f"⚠️ Could not check index status: {e}")
        print(f"   Proceeding with data source creation...")


# Create Confluence Data Source
def create_confluence_datasource(
    app_id, index_id, datasource_name, secret_arn, role_arn, site_id
):
    qbusiness = boto3.client("qbusiness", region_name=region)

    try:
        print(f"🔗 Creating Confluence data source: {datasource_name}")

        # Confluence data source configuration
        # This configuration crawls all spaces and content types (pages, blogs, attachments)
        # To customize: modify repositoryConfigurations to include/exclude specific spaces,
        # add content filters, or adjust field mappings for your use case
        datasource_config = {
            "type": "CONFLUENCE",
            "secretArn": secret_arn,
            "connectionConfiguration": {
                "repositoryEndpointMetadata": {
                    "hostUrl": f"https://{confluence_domain}",
                    "type": "SAAS",
                    "authType": "OAuth2",
                }
            },
            "repositoryConfigurations": {
                "page": {
                    "fieldMappings": [
                        {
                            "dataSourceFieldName": "author",
                            "indexFieldName": "_authors",
                            "indexFieldType": "STRING_LIST",
                        },
                        {
                            "dataSourceFieldName": "createdDate",
                            "dateFieldFormat": "yyyy-MM-dd'T'HH:mm:ss'Z'",
                            "indexFieldName": "_created_at",
                            "indexFieldType": "DATE",
                        },
                        {
                            "dataSourceFieldName": "modifiedDate",
                            "dateFieldFormat": "yyyy-MM-dd'T'HH:mm:ss'Z'",
                            "indexFieldName": "_last_updated_at",
                            "indexFieldType": "DATE",
                        },
                        {
                            "dataSourceFieldName": "url",
                            "indexFieldName": "_source_uri",
                            "indexFieldType": "STRING",
                        },
                    ]
                },
                "blog": {
                    "fieldMappings": [
                        {
                            "dataSourceFieldName": "author",
                            "indexFieldName": "_authors",
                            "indexFieldType": "STRING_LIST",
                        },
                        {
                            "dataSourceFieldName": "publishedDate",
                            "dateFieldFormat": "yyyy-MM-dd'T'HH:mm:ss'Z'",
                            "indexFieldName": "_created_at",
                            "indexFieldType": "DATE",
                        },
                        {
                            "dataSourceFieldName": "url",
                            "indexFieldName": "_source_uri",
                            "indexFieldType": "STRING",
                        },
                    ]
                },
                "space": {
                    "fieldMappings": [
                        {
                            "dataSourceFieldName": "url",
                            "indexFieldName": "_source_uri",
                            "indexFieldType": "STRING",
                        }
                    ]
                },
            },
            "additionalProperties": {
                "isCrawlAcl": False,
                "isCrawlPage": True,
                "isCrawlBlog": True,
                "isCrawlPageAttachment": True,
                "isCrawlBlogAttachment": True,
                "isCrawlPageComment": True,
                "isCrawlBlogComment": True,
                "isCrawlPersonalSpace": True,
                "isCrawlArchivedSpace": True,
                "isCrawlArchivedPage": True,
                "maxFileSizeInMegaBytes": "50",
                "fieldForUserId": "uuid",
                "isRotateSecret": True,
                "enableDeletionProtection": False,
                "deletionProtectionThreshold": "0",
            },
            "syncMode": "FORCED_FULL_CRAWL",
            "version": "1.0.0",
            "enableIdentityCrawler": False,
        }

        datasource_response = qbusiness.create_data_source(
            applicationId=app_id,
            indexId=index_id,
            displayName=datasource_name,
            description="Confluence Cloud data source",
            configuration=datasource_config,
            roleArn=role_arn,
        )

        datasource_id = datasource_response["dataSourceId"]
        print(f"✅ Data source created: {datasource_id}")
        return datasource_id

    except Exception as e:
        print(f"❌ Failed to create data source: {str(e)}")
        return None


# Check for required variables with detailed diagnostics
print("🔍 Checking required variables for data source creation...")
required_vars = [
    "application_id",
    "index_id",
    "secret_arn",
    "datasource_role_arn",
    "site_id",
]
missing_vars = []
available_vars = []

for var_name in required_vars:
    if var_name in locals() and locals()[var_name] is not None:
        available_vars.append(var_name)
        print(
            f"   ✅ {var_name}: {str(locals()[var_name])[:50]}{'...' if len(str(locals()[var_name])) > 50 else ''}"
        )
    else:
        missing_vars.append(var_name)
        print(f"   ❌ {var_name}: Not available")

if not missing_vars:
    print("\n✅ All required variables are available!")

    # Create the data source
    datasource_id = create_confluence_datasource(
        application_id,
        index_id,
        datasource_name,
        secret_arn,
        datasource_role_arn,
        site_id,
    )

    if datasource_id:
        print(f"\n🎉 Data Source Created Successfully!")
        print(f"Data Source ID: {datasource_id}")
        print("\n📋 Next Steps:")
        print("1. Proceed to create the retriever")
        print("2. Create the web experience")
        print("3. Start the data source sync")
    else:
        print("\n❌ Failed to create data source")
        print("Check the error messages above for details")
else:
    print(
        f"\n⚠️ Missing {len(missing_vars)} required variable(s): {', '.join(missing_vars)}"
    )
    print("\n📋 To fix this, make sure you've successfully run:")
    if "application_id" in missing_vars:
        print("   - Step 9: Create Q Business Application")
    if "index_id" in missing_vars:
        print("   - Step 10: Create Q Business Index (and wait for it to be ACTIVE)")
    if "secret_arn" in missing_vars:
        print("   - Step 7: Store Tokens in AWS Secrets Manager")
    if "datasource_role_arn" in missing_vars:
        print("   - Step 8: Create IAM Role for Q Business")
    if "site_id" in missing_vars:
        print("   - Step 6: Test Confluence API Access")
    print("\n💡 Each step must complete successfully before proceeding to the next.")

## Step 12: Create Q Business Retriever

A retriever enables semantic search capabilities in Q Business. It connects to your index and allows the AI to find relevant content based on the meaning of user questions, not just keyword matching. This is what powers the intelligent responses in your chat interface.

In [ ]:
# Create Q Business Retriever
def create_qbusiness_retriever(app_id, index_id, retriever_name="Confluence-Retriever"):
    qbusiness = boto3.client("qbusiness", region_name=region)

    try:
        print(f"🔍 Creating retriever: {retriever_name}")

        retriever_response = qbusiness.create_retriever(
            applicationId=app_id,
            displayName=retriever_name,
            type="NATIVE_INDEX",
            configuration={"nativeIndexConfiguration": {"indexId": index_id}},
        )

        retriever_id = retriever_response["retrieverId"]
        print(f"✅ Retriever created: {retriever_id}")
        return retriever_id

    except Exception as e:
        print(f"❌ Failed to create retriever: {str(e)}")
        return None


# Check for required variables
print("🔍 Checking required variables for retriever creation...")
required_vars = ["application_id", "index_id"]
missing_vars = []

for var_name in required_vars:
    if var_name in locals() and locals()[var_name] is not None:
        print(
            f"   ✅ {var_name}: {str(locals()[var_name])[:50]}{'...' if len(str(locals()[var_name])) > 50 else ''}"
        )
    else:
        missing_vars.append(var_name)
        print(f"   ❌ {var_name}: Not available")

if not missing_vars:
    print("\n✅ All required variables are available!")

    # Create the retriever
    retriever_id = create_qbusiness_retriever(application_id, index_id)

    if retriever_id:
        print(f"\n🎉 Retriever Created Successfully!")
        print(f"Retriever ID: {retriever_id}")
    else:
        print("\n❌ Failed to create retriever")
else:
    print(
        f"\n⚠️ Missing {len(missing_vars)} required variable(s): {', '.join(missing_vars)}"
    )
    print("\n📋 To fix this, make sure you've successfully run:")
    if "application_id" in missing_vars:
        print("   - Step 9: Create Q Business Application")
    if "index_id" in missing_vars:
        print("   - Step 10: Create Q Business Index (and wait for it to be ACTIVE)")

## Step 13: Create Q Business Web Experience

The web experience provides a user-friendly chat interface where your team can ask questions about Confluence content. Users can have natural conversations with the AI, which will search through your indexed content and provide accurate, source-cited responses. This is the main interface your users will interact with.

In [ ]:
# Create Q Business Web Experience
def create_web_experience(app_id, web_experience_title="Confluence Q Business Chat"):
    """Create a Q Business web experience for the application"""
    qbusiness = boto3.client("qbusiness", region_name=region)

    try:
        print(f"🌐 Creating Q Business Web Experience: {web_experience_title}")

        # Create web experience
        web_response = qbusiness.create_web_experience(
            applicationId=app_id,
            title=web_experience_title,
            subtitle="Chat with your Confluence content using Amazon Q Business",
            welcomeMessage="Welcome! I can help you find information from your Confluence spaces. What would you like to know?",
            roleArn=datasource_role_arn,
        )

        web_experience_id = web_response["webExperienceId"]
        print(f"✅ Web Experience created: {web_experience_id}")

        # Wait for activation
        print("⏳ Waiting for web experience to become active...")

        for attempt in range(10):
            try:
                web_status = qbusiness.get_web_experience(
                    applicationId=app_id, webExperienceId=web_experience_id
                )

                status = web_status.get("status", "UNKNOWN")
                endpoint = web_status.get("defaultEndpoint", "Not available yet")

                print(f"   Status: {status}")

                if status == "ACTIVE":
                    print(f"✅ Web Experience is active!")
                    print(f"🔗 URL: {endpoint}")
                    return web_experience_id, endpoint
                elif status in ["FAILED"]:
                    print(f"❌ Web Experience failed: {status}")
                    return None, None
                else:
                    time.sleep(30)

            except Exception as e:
                print(f"   Error checking status: {e}")
                time.sleep(30)

        # Return final status even if not active
        try:
            final_status = qbusiness.get_web_experience(
                applicationId=app_id, webExperienceId=web_experience_id
            )
            return web_experience_id, final_status.get(
                "defaultEndpoint", "Check AWS Console"
            )
        except:
            return web_experience_id, "Check AWS Console"

    except Exception as e:
        print(f"❌ Failed to create web experience: {e}")
        return None, None


# Create the web experience
print("🌐 Creating Q Business Web Experience...")
print("This will be available immediately for users to access!")
print()

if (
    "application_id" in locals()
    and application_id
    and "datasource_role_arn" in locals()
    and datasource_role_arn
):
    web_experience_id, web_url = create_web_experience(
        app_id=application_id, web_experience_title="Confluence Q Business Chat"
    )

    if web_experience_id:
        print(f"\n🎉 SUCCESS! Web Experience Created")
        print(f"Web Experience ID: {web_experience_id}")
        print(f"URL: {web_url}")
        print()
        print("📋 Next Steps:")
        print("1. Users can now access the Q Business chat interface")
        print("2. Run the data source sync to populate content")
        print("3. Test the integration by asking questions")
    else:
        print("\n❌ Failed to create web experience")
        print("Check the error messages above")
else:
    print("⚠️ No application_id available. Complete previous steps first.")

## Step 14: Start Data Source Synchronization

Data source synchronization is the process where Q Business crawls your Confluence spaces, extracts content, and indexes it for search. The initial sync can take time depending on your content volume. You can monitor sync progress in the Q Business console and set up scheduled syncs to keep content up-to-date.

In [ ]:
# Start data source synchronization
def start_datasource_sync(app_id, datasource_id, index_id):
    """Start Q Business data source sync job"""
    qbusiness = boto3.client("qbusiness", region_name=region)

    try:
        print(f"🔄 Starting sync job...")
        print(f"   Application ID: {app_id}")
        print(f"   Index ID: {index_id}")
        print(f"   Data Source ID: {datasource_id}")

        response = qbusiness.start_data_source_sync_job(
            applicationId=app_id, indexId=index_id, dataSourceId=datasource_id
        )

        execution_id = response["executionId"]
        print(f"✅ Sync job started successfully!")
        print(f"Execution ID: {execution_id}")
        return execution_id

    except Exception as e:
        print(f"❌ Failed to start sync job: {str(e)}")
        return None


# Check for required variables and start sync
print("🔍 Checking for required variables...")

# Check each variable individually
missing_vars = []
if "application_id" not in locals() or not application_id:
    missing_vars.append("application_id")
if "index_id" not in locals() or not index_id:
    missing_vars.append("index_id")
if "datasource_id" not in locals() or not datasource_id:
    missing_vars.append("datasource_id")

if missing_vars:
    print(f"⚠️ Missing required variables: {', '.join(missing_vars)}")
    print("\n📋 To fix this, make sure you've successfully run:")
    print("- Step 9: Create Q Business Application")
    print("- Step 10: Create Index and Data Source")
else:
    print("✅ All required variables found!")
    print()

    # Start the sync
    execution_id = start_datasource_sync(application_id, datasource_id, index_id)

    if execution_id:
        print(f"\n🎉 SUCCESS! Data Source Sync Started")
        print(f"Execution ID: {execution_id}")
        print()
        print("📋 What's happening now:")
        print("1. Q Business is connecting to your Confluence instance")
        print("2. Content is being crawled and indexed")
        print("3. This process runs in the background")
        print("4. You can monitor progress in the AWS Console")
    else:
        print("\n❌ Failed to start data source sync")
        print("Check the error messages above for details.")

## 🎉 Setup Complete!

If you've successfully run all the steps above, you now have:

✅ **Confluence OAuth tokens** stored securely in AWS Secrets Manager  
✅ **Q Business application** created and configured  
✅ **Confluence data source** connected to your instance  
✅ **Web experience** available for users to access  
✅ **Data synchronization** running in the background  

### Next Steps:

1. **Monitor the sync**: Check the AWS Console to see sync progress
2. **Test the integration**: Use the web experience URL to ask questions
3. **Configure permissions**: Set up user access as needed

### Resources:

- [Amazon Q Business User Guide](https://docs.aws.amazon.com/amazonq/latest/qbusiness-ug/)
- [AWS Q Business Console](https://console.aws.amazon.com/qbusiness/)